In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import os
from tqdm import tqdm
import glob

In [2]:
import os

def get_npy_files(root_path):
    npy_files = []

    for dirpath, dirnames, filenames in os.walk(root_path):
        for filename in filenames:
            if filename.endswith('.npz'):
                npy_files.append(os.path.join(dirpath, filename))

    return npy_files
ROOT = "/app/dataset/npz_all/npz"
all_npy_files = get_npy_files(ROOT)
df = pd.DataFrame(all_npy_files)
df.columns = ["path"]
df

,path
0,/app/dataset/npz_all/npz/layout/nlp/default/te...
1,/app/dataset/npz_all/npz/layout/nlp/default/te...
2,/app/dataset/npz_all/npz/layout/nlp/default/te...
3,/app/dataset/npz_all/npz/layout/nlp/default/te...
4,/app/dataset/npz_all/npz/layout/nlp/default/te...
...,...
7863,/app/dataset/npz_all/npz/tile/xla/valid/unet_3...
7864,/app/dataset/npz_all/npz/tile/xla/valid/unet_3...
7865,/app/dataset/npz_all/npz/tile/xla/valid/unet_3...
7866,/app/dataset/npz_all/npz/tile/xla/valid/unet_3...


In [3]:
def find_split(x):
    if "train" in x:
        return "train"
    elif "valid" in x:
        return "valid"
    elif "test" in x:
        return "test"
    return None
def find_configtype(x):
    if "default" in x:
        return "default"
    elif "random" in x:
        return "random"
    return "istile"
def find_modeltype(x):
    if "nlp" in x:
        return "nlp"
    elif "xla" in x:
        return "xla"
    return None
def find_grapthtype(x):
    if "layout" in x:
        return "layout"
    elif "tile" in x:
        return "tile"
    return None
df["split"] = df.path.apply(lambda x : find_split(x))
df["config_type"] = df.path.apply(lambda x : find_configtype(x))
df["modeltype"] = df.path.apply(lambda x : find_modeltype(x))
df["graphtype"] = df.path.apply(lambda x : find_grapthtype(x))


In [4]:
# num_edges = []
# num_nodes = []
# num_configs = []
# layout_df = df[df.graphtype=="layout"].reset_index(drop=True)
# for d in tqdm(layout_df.path):
#     data = dict(np.load(d))
#     num_edges.append(len(data["edge_index"]))
#     num_nodes.append(len(data["node_feat"]))
#     num_configs.append(len(data["config_runtime"]))
# layout_df["num_edges"] = num_edges
# layout_df["num_nodes"] = num_nodes
# layout_df["num_configs"] = num_configs
# layout_df.to_csv("/app/nn-runtime-network/workdir/csvs/layout_info.csv",index=False)

In [5]:
import os
# df = layout_df
for spl in df.split.unique():
    
    # Replace these with your list of file paths
    if spl=="train" and True:
        file_paths = df[(df.split==spl)&(df.graphtype=="layout")].path.tolist()
    else:
        file_paths = df[(df.split==spl)&(df.graphtype=="layout")].path.tolist()

    folder_path = f"/app/dataset/various_splits/all_layout/{spl}"
    os.makedirs(folder_path, exist_ok=True)

    # Create symbolic links in the folder
    for file_path in file_paths:
        # Extract the file name from the file path
        file_name = "___".join(file_path.split("/")[-4:])
        
        # Create a symlink in the 'train' folder pointing to the original file
        symlink_path = os.path.join(folder_path, file_name)
        
        # Create the symlink
        os.symlink(file_path, symlink_path)

    print(f'Symlinks to {len(file_paths)} files created in the "{folder_path}" folder.')


Symlinks to 50 files created in the "/app/dataset/various_splits/all_layout/test" folder.
Symlinks to 565 files created in the "/app/dataset/various_splits/all_layout/train" folder.
Symlinks to 24 files created in the "/app/dataset/various_splits/all_layout/valid" folder.


# All tiles

In [5]:
import os
# df = layout_df
for spl in df.split.unique():
    
    # Replace these with your list of file paths
    if spl=="train" and True:
        file_paths = df[(df.split==spl)&(df.graphtype=="tile")].path.tolist()
    else:
        file_paths = df[(df.split==spl)&(df.graphtype=="tile")].path.tolist()

    folder_path = f"/app/dataset/various_splits/all_tile/{spl}"
    os.makedirs(folder_path, exist_ok=True)

    # Create symbolic links in the folder
    for file_path in file_paths:
        # Extract the file name from the file path
        file_name = "___".join(file_path.split("/")[-4:])
        
        # Create a symlink in the 'train' folder pointing to the original file
        symlink_path = os.path.join(folder_path, file_name)
        
        # Create the symlink
        os.symlink(file_path, symlink_path)

    print(f'Symlinks to {len(file_paths)} files created in the "{folder_path}" folder.')


Symlinks to 844 files created in the "/app/dataset/various_splits/all_tile/test" folder.
Symlinks to 6089 files created in the "/app/dataset/various_splits/all_tile/train" folder.
Symlinks to 296 files created in the "/app/dataset/various_splits/all_tile/valid" folder.
